### Setup

Import the necessary libraries. NetCDF for creating and 
manipulating the netcdf file, and numpy for operations on 
the data.

In [ ]:
import netCDF4     
import numpy as np

Now, start by opening the file.

In [ ]:
try:
    ncfile.close()
except:
    pass
ncfile = netCDF4.Dataset("new-tem-input.nc", mode="w", format='NETCDF4')
print(ncfile)

### Create Dimensions
Setup the dimensions for the file. Basically a data "cube" of time and (Y,X) position in the grid.

In [ ]:
# Dimensions for the file.
Y = ncfile.createDimension('X', 10)
X = ncfile.createDimension('Y', 10)
time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).

for dim in list(ncfile.dimensions.items()):
    print(dim)

### Create Coordinate Variables


In [ ]:
# Coordinate Variables
X = ncfile.createVariable('X', np.int, ('X',))
Y = ncfile.createVariable('Y', np.int, ('Y',))
lat = ncfile.createVariable('lat', np.float32, ('Y', 'X',))
lon = ncfile.createVariable('lon', np.float32, ('Y', 'X',))

### Create Data Variables

#### Climate

In [ ]:
co2 = ncfile.createVariable('co2', np.float32, ('time')) # actually year
temp_air = ncfile.createVariable('tair', np.float32, ('time', 'Y', 'X',))
precip = ncfile.createVariable('precip', np.float32, ('time', 'Y', 'X',))
nirr = ncfile.createVariable('nirr', np.float32, ('time', 'Y', 'X',))
vapor_press = ncfile.createVariable('vapor_press', np.float32, ('time', 'Y', 'X',))

#### Soil
Here is a quick overview of how soil is imagined:

    (snow)
    (moss) green and brown? maybe green is pfts?
    =========================
    organic layer
    ---------
    0..n shallow (fibric)
    ---------
    0..n layers deep (humic)
    =========================
    Mineral layer
    (top) bottom of organic to -25cm depth
    ----------
    (bottom) from -25cm to -100cm or to bedrock

In [ ]:
# Data will come from tifs on Helene's computer
# Read as "% of material that is sand in the upper mineral layer"
sand_top = ncfile.createVariable('sand_top', np.float32, ('Y','X')) 
sand_bottom = ncfile.createVariable('sand_bottom', np.float32, ('Y','X')) 
silt_top = ncfile.createVariable('silt_top', np.float32, ('Y','X')) 
silt_bottom = ncfile.createVariable('silt_bottom', np.float32, ('Y','X')) 
clay_top = ncfile.createVariable('clay_top', np.float32, ('Y','X')) 
clay_bottom = ncfile.createVariable('clay_bottom', np.float32, ('Y','X'))

In [ ]:
# BOOLEAN - 0 is dry, 1 is wet (poorly drained)
# 1km res, from cost function of slope (defined on HYDRO unit, level 8)
drainage_type = ncfile.createVariable('drainage_type', np.int, ('Y', 'X'))

# Vegetation
veg_class = ncfile.createVariable('veg_class', np.int, ('Y', 'X',))

# FIRE

# 1) for eq and sp we need a fire recurrance interval
# varies with space and *NOT* time
# generated by some statictical munging of the SNAP Fire History tif files
# will result in a single band tiff
fri = ncfile.createVariable('fri', np.int, ('Y','X'))

# 2) for tr and sc FRI is based on ALF outputs or historical records (for AK from 1952)
# year_of_burn - explicit replacement for FRI in tr and sc (maybe from ALF)
fire_year_vector = ncfile.createVLType(np.int, 'fire_year_vector')
fire_years = ncfile.createVariable('fire_years', fire_year_vector, ('Y','X'))

# Size - measured in area of burn varies with space and time, but time is 
# encoded in this vector.
fire_sizes = ncfile.createVariable('fire_sizes', fire_year_vector, ('Y','X'))

> NOTE: As of conversation with Ruth, Helene and Tobey, 8/28/2015, 
we decided to add to this slightly
>
> Basically during the large refactor process, the way the fire module
> is reshaping, it might make sense to do add a "fire_month" vector
> that holds the month the fire should occur. This enables getting
> rid of the "season" classification previously used in the model.
> Stay tuned...

### Populate with random sample data
Start with the basic grid coordinates. Basically just index values.

In [ ]:
nX = len(X); nY = len(Y); ntimes = 100
X[:] = list(range(0, nX))
Y[:] = list(range(0, nY))

Now fill in some sample lat and lon values for the grid.

In [ ]:
print len(np.linspace(63.5, 65.2, 10))
print lat.shape
ncfile.close()

!ncdump -h "new-tem-input.nc"

In [ ]:
print(lat)
# lat[:] = np.linspace(63.5, 65.2, 10)
# lon[:] = np.linspace(-147.5, -145.9, 10)

In [ ]:
co2[:] = np.linspace(350, 400, ntimes)

In [ ]:
temp_air[:] = np.random.uniform(low=-20.0, high=35.0, size=(ntimes, nlats, nlons))

## Notes

We will likely want seperate files for

* soil (and drainage)
* climate
* fire 
* veg
* run mask

Having seperate files introduces some overhead in terms of 
keeping spatial information with each file, but for now we feel 


A valid input set requires that each file has data for the same grid cells.




most importatn outputs
vegC accumulated
NPP
active layerdepth
soil temperature


